## Distributed Computing  Project - Task 2
## Tom Kovalcik
## CDs and Vinyl Amazon Reviews Dataset

### Imports

In [ ]:
import pandas as pd
import pyspark
import os
import types
import datetime
import json
from pathlib import Path
from pyspark.sql import SparkSession
import json

### Extract first 5000 lines from original dataset and save as separate file

In [ ]:
cat_name = "CDs_and_Vinyl"
sample_size = 5000

input_path = Path(f"../../../{cat_name}.jsonl")
output_path = Path(f"../../data_samples/{cat_name}_SAMPLE.jsonl")


with input_path.open("r", encoding="utf-8") as file_obj:
    with output_path.open("w", encoding="utf-8") as out_file_obj:
        for i, line in enumerate(file_obj):
            if i <= sample_size:
                out_file_obj.write(line)
            else:
                break
print(f"{sample_size} lines written to {output_path}")

5000 lines written to ../../data_samples/CDs_and_Vinyl_SAMPLE.jsonl


In [ ]:

spark = SparkSession.builder.appName("CDsAndVinylSample").getOrCreate()
sc = spark.sparkContext

path = "/mnt/data/CDs_and_Vinyl_SAMPLE.jsonl"

# Each element of this RDD is one line (a JSON string)
raw_rdd = sc.textFile(path)

# Optional: convert each JSON string -> Python dict
records_rdd = raw_rdd.map(json.loads)

# Quick sanity checks:
print("Number of records:", records_rdd.count())
print("First record:", records_rdd.first())